# একটি শ্রেণীবিন্যাস মডেল তৈরি করুন: সুস্বাদু এশিয়ান এবং ভারতীয় খাবার


## রান্নার শ্রেণীবিন্যাস ২

এই দ্বিতীয় শ্রেণীবিন্যাস পাঠে, আমরা শ্রেণীবদ্ধ তথ্য বিশ্লেষণের `আরও উপায়` অন্বেষণ করব। আমরা একটি শ্রেণীবিন্যাসকারী বেছে নেওয়ার প্রভাব সম্পর্কেও শিখব।

### [**পাঠের আগে কুইজ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **প্রয়োজনীয়তা**

আমরা ধরে নিচ্ছি যে আপনি আগের পাঠগুলি সম্পন্ন করেছেন, কারণ আমরা পূর্বে শেখা কিছু ধারণা এখানে ব্যবহার করব।

এই পাঠের জন্য, আমাদের নিম্নলিখিত প্যাকেজগুলোর প্রয়োজন হবে:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) হলো [R প্যাকেজগুলোর একটি সংগ্রহ](https://www.tidyverse.org/packages), যা ডেটা সায়েন্সকে দ্রুত, সহজ এবং আরও মজাদার করে তোলে!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) ফ্রেমওয়ার্ক হলো মডেলিং এবং মেশিন লার্নিংয়ের জন্য [প্যাকেজগুলোর একটি সংগ্রহ](https://www.tidymodels.org/packages/)।

-   `themis`: [themis প্যাকেজ](https://themis.tidymodels.org/) ভারসাম্যহীন ডেটার সাথে কাজ করার জন্য অতিরিক্ত রেসিপি ধাপ সরবরাহ করে।

আপনি এগুলো নিম্নলিখিতভাবে ইনস্টল করতে পারেন:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

অথবা, নিচের স্ক্রিপ্টটি পরীক্ষা করে দেখবে যে এই মডিউলটি সম্পন্ন করার জন্য প্রয়োজনীয় প্যাকেজগুলো আপনার কাছে আছে কিনা এবং যদি না থাকে তবে সেগুলো ইনস্টল করবে।


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. একটি শ্রেণীবিন্যাস মানচিত্র**

আমাদের [পূর্ববর্তী পাঠে](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), আমরা এই প্রশ্নের উত্তর খুঁজতে চেষ্টা করেছি: কীভাবে আমরা একাধিক মডেলের মধ্যে নির্বাচন করব? অনেকাংশে, এটি ডেটার বৈশিষ্ট্য এবং আমরা যে সমস্যাটি সমাধান করতে চাই তার ধরণ (যেমন শ্রেণীবিন্যাস বা রিগ্রেশন) এর উপর নির্ভর করে।

আগে, আমরা Microsoft's চিট শিট ব্যবহার করে ডেটা শ্রেণীবিন্যাস করার বিভিন্ন বিকল্প সম্পর্কে শিখেছি। Python-এর Machine Learning ফ্রেমওয়ার্ক, Scikit-learn, একটি অনুরূপ কিন্তু আরও বিস্তারিত চিট শিট প্রদান করে যা আপনার শ্রেণীবিন্যাসকারীদের (যা "এস্টিমেটর" নামেও পরিচিত) আরও সংকুচিত করতে সাহায্য করতে পারে:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> টিপ: [এই মানচিত্রটি অনলাইনে দেখুন](https://scikit-learn.org/stable/tutorial/machine_learning_map/) এবং পথ ধরে ক্লিক করে ডকুমেন্টেশন পড়ুন।  
>  
> [Tidymodels রেফারেন্স সাইট](https://www.tidymodels.org/find/parsnip/#models) বিভিন্ন ধরনের মডেলের চমৎকার ডকুমেন্টেশন সরবরাহ করে।  

### **পরিকল্পনা** 🗺️  

এই মানচিত্রটি আপনার ডেটা সম্পর্কে পরিষ্কার ধারণা থাকলে খুবই সহায়ক, কারণ আপনি এর পথ ধরে একটি সিদ্ধান্তে পৌঁছাতে পারেন:  

-   আমাদের কাছে \>৫০টি নমুনা রয়েছে  

-   আমরা একটি শ্রেণী পূর্বানুমান করতে চাই  

-   আমাদের কাছে লেবেলযুক্ত ডেটা রয়েছে  

-   আমাদের কাছে ১ লক্ষের কম নমুনা রয়েছে  

-   ✨ আমরা একটি Linear SVC বেছে নিতে পারি  

-   যদি এটি কাজ না করে, যেহেতু আমাদের কাছে সংখ্যাসূচক ডেটা রয়েছে  

    -   আমরা ✨ KNeighbors Classifier চেষ্টা করতে পারি  

        -   যদি এটি কাজ না করে, ✨ SVC এবং ✨ Ensemble Classifiers চেষ্টা করুন  

এটি অনুসরণ করার জন্য একটি খুব সহায়ক পথ। এখন, চলুন [tidymodels](https://www.tidymodels.org/) মডেলিং ফ্রেমওয়ার্ক ব্যবহার করে সরাসরি কাজে নেমে পড়ি: এটি R প্যাকেজগুলির একটি সঙ্গতিপূর্ণ এবং নমনীয় সংগ্রহ যা ভালো পরিসংখ্যান চর্চাকে উৎসাহিত করার জন্য তৈরি করা হয়েছে 😊।  

## ২. ডেটা ভাগ করুন এবং অসম ডেটাসেট পরিচালনা করুন।  

আমাদের আগের পাঠ থেকে আমরা শিখেছি যে আমাদের রান্নার ধরনগুলির মধ্যে কিছু সাধারণ উপাদান ছিল। এছাড়াও, রান্নার ধরনগুলির সংখ্যায় বেশ অসম বণ্টন ছিল।  

আমরা এগুলো মোকাবিলা করব  

-   `dplyr::select()` ব্যবহার করে এমন সবচেয়ে সাধারণ উপাদান বাদ দিয়ে, যা ভিন্ন রান্নার ধরনগুলির মধ্যে বিভ্রান্তি সৃষ্টি করে।  

-   একটি `recipe` ব্যবহার করে, যা ডেটাকে প্রি-প্রসেস করে মডেলিংয়ের জন্য প্রস্তুত করে এবং একটি `over-sampling` অ্যালগরিদম প্রয়োগ করে।  

আমরা আগের পাঠে উপরোক্ত বিষয়গুলি ইতিমধ্যেই দেখেছি, তাই এটি সহজ হবে 🥳!  


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### অসমতল ডেটা নিয়ে কাজ করা

অসমতল ডেটা প্রায়ই মডেলের কার্যকারিতার উপর নেতিবাচক প্রভাব ফেলে। অনেক মডেল তখনই সেরা পারফর্ম করে যখন পর্যবেক্ষণের সংখ্যা সমান হয়, এবং তাই অসমতল ডেটার ক্ষেত্রে তারা প্রায়ই সমস্যায় পড়ে।

অসমতল ডেটা সেটের সাথে কাজ করার দুটি প্রধান উপায় রয়েছে:

-   সংখ্যালঘু শ্রেণিতে পর্যবেক্ষণ যোগ করা: `ওভার-স্যাম্পলিং`, যেমন SMOTE অ্যালগরিদম ব্যবহার করে যা সংখ্যালঘু শ্রেণির নতুন উদাহরণ কৃত্রিমভাবে তৈরি করে এই কেসগুলোর নিকটতম প্রতিবেশীদের ব্যবহার করে।

-   সংখ্যাগরিষ্ঠ শ্রেণি থেকে পর্যবেক্ষণ সরিয়ে ফেলা: `আন্ডার-স্যাম্পলিং`

আমাদের আগের পাঠে, আমরা দেখিয়েছিলাম কীভাবে একটি `রেসিপি` ব্যবহার করে অসমতল ডেটা সেটের সাথে কাজ করা যায়। একটি রেসিপিকে একটি নকশা হিসেবে ভাবা যেতে পারে যা বর্ণনা করে ডেটা সেটে কী ধাপগুলো প্রয়োগ করা উচিত যাতে এটি ডেটা বিশ্লেষণের জন্য প্রস্তুত হয়। আমাদের ক্ষেত্রে, আমরা আমাদের `ট্রেনিং সেট`-এর জন্য আমাদের রান্নার ধরনগুলোর মধ্যে সমান বণ্টন চাই। চলুন শুরু করা যাক।


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

এখন আমরা মডেল প্রশিক্ষণের জন্য প্রস্তুত 👩‍💻👨‍💻!

## ৩. বহুগুণিত রিগ্রেশন মডেলের বাইরে

আমাদের আগের পাঠে, আমরা বহুগুণিত রিগ্রেশন মডেল নিয়ে আলোচনা করেছি। চলুন আরও কিছু নমনীয় শ্রেণীবিন্যাস মডেল অন্বেষণ করি।

### সাপোর্ট ভেক্টর মেশিনস

শ্রেণীবিন্যাসের ক্ষেত্রে, `সাপোর্ট ভেক্টর মেশিনস` একটি মেশিন লার্নিং পদ্ধতি যা শ্রেণীগুলিকে "সেরা"ভাবে আলাদা করার জন্য একটি *হাইপারপ্লেন* খুঁজে বের করার চেষ্টা করে। চলুন একটি সহজ উদাহরণ দেখি:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ ক্লাসগুলোকে আলাদা করে না। H2~ আলাদা করে, তবে খুব ছোট মার্জিন দিয়ে। H3~ সর্বাধিক মার্জিন দিয়ে ক্লাসগুলোকে আলাদা করে।

#### লিনিয়ার সাপোর্ট ভেক্টর ক্লাসিফায়ার

সাপোর্ট-ভেক্টর ক্লাস্টারিং (SVC) হলো সাপোর্ট-ভেক্টর মেশিনস পরিবারের একটি শাখা, যা মেশিন লার্নিং প্রযুক্তির অন্তর্ভুক্ত। SVC-তে হাইপারপ্লেন এমনভাবে নির্বাচন করা হয় যাতে `বেশিরভাগ` ট্রেনিং অবজারভেশন সঠিকভাবে আলাদা করা যায়, তবে কিছু অবজারভেশন `ভুলভাবে শ্রেণীবদ্ধ` হতে পারে। কিছু পয়েন্টকে ভুল দিকে থাকার অনুমতি দিয়ে, SVM আউটলায়ারগুলোর প্রতি আরও সহনশীল হয় এবং নতুন ডেটার ক্ষেত্রে আরও ভালো জেনারালাইজেশন প্রদান করে। এই ভায়োলেশন নিয়ন্ত্রণ করার প্যারামিটারকে `কস্ট` বলা হয়, যার ডিফল্ট মান ১ (দেখুন `help("svm_poly")`)।

চলুন একটি লিনিয়ার SVC তৈরি করি, যেখানে পলিনোমিয়াল SVM মডেলে `degree = 1` সেট করা হয়েছে।


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

এখন যেহেতু আমরা প্রিপ্রসেসিং ধাপ এবং মডেল স্পেসিফিকেশনকে একটি *workflow*-এ সংযুক্ত করেছি, আমরা লিনিয়ার SVC প্রশিক্ষণ দিতে এবং এর ফলাফল মূল্যায়ন করতে এগিয়ে যেতে পারি। পারফরম্যান্স মেট্রিকের জন্য, চলুন একটি মেট্রিক সেট তৈরি করি যা মূল্যায়ন করবে: `accuracy`, `sensitivity`, `Positive Predicted Value` এবং `F Measure`।

> `augment()` প্রদত্ত ডেটাতে পূর্বাভাসের জন্য কলাম(গুলো) যোগ করবে।


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### সাপোর্ট ভেক্টর মেশিন

সাপোর্ট ভেক্টর মেশিন (SVM) হল সাপোর্ট ভেক্টর ক্লাসিফায়ারের একটি সম্প্রসারণ, যা শ্রেণিগুলোর মধ্যে একটি অ-রৈখিক সীমানা সামঞ্জস্য করার জন্য তৈরি করা হয়েছে। মূলত, SVMs *কর্নেল ট্রিক* ব্যবহার করে বৈশিষ্ট্য স্থানকে বড় করে তোলে, যাতে শ্রেণিগুলোর মধ্যে অ-রৈখিক সম্পর্কের সাথে মানিয়ে নেওয়া যায়। SVMs দ্বারা ব্যবহৃত একটি জনপ্রিয় এবং অত্যন্ত নমনীয় কর্নেল ফাংশন হল *রেডিয়াল বেসিস ফাংশন।* চলুন দেখি এটি আমাদের ডেটার উপর কীভাবে কাজ করে।


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

অনেক ভালো 🤩!

> ✅ দয়া করে দেখুন:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> আরও পড়ার জন্য।

### নিকটতম প্রতিবেশী শ্রেণিবিন্যাসকারী

*K*-নিকটতম প্রতিবেশী (KNN) একটি অ্যালগরিদম যেখানে প্রতিটি পর্যবেক্ষণকে অন্যান্য পর্যবেক্ষণের সাথে তার *সাদৃশ্য* এর উপর ভিত্তি করে পূর্বানুমান করা হয়।

চলুন আমাদের ডেটার উপর এটি প্রয়োগ করি।


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

মডেলটি মনে হচ্ছে খুব ভালো কাজ করছে না। সম্ভবত মডেলের আর্গুমেন্ট পরিবর্তন করা (দেখুন `help("nearest_neighbor")`) মডেলের কার্যকারিতা উন্নত করবে। অবশ্যই এটি চেষ্টা করে দেখুন।

> ✅ দয়া করে দেখুন:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> আরও জানতে *K*-Nearest Neighbors ক্লাসিফায়ার সম্পর্কে।

### এনসেম্বল ক্লাসিফায়ার

এনসেম্বল অ্যালগরিদমগুলি একাধিক বেস এস্টিমেটরকে একত্রিত করে একটি অপ্টিমাল মডেল তৈরি করে, যা করা হয় নিম্নলিখিত পদ্ধতিতে:

`bagging`: বেস মডেলগুলির একটি সংগ্রহে একটি *গড় ফাংশন* প্রয়োগ করা

`boosting`: একের পর এক মডেল তৈরি করা যা একে অপরের উপর ভিত্তি করে ভবিষ্যদ্বাণী করার কার্যকারিতা উন্নত করে।

চলুন শুরু করি একটি র‍্যান্ডম ফরেস্ট মডেল দিয়ে চেষ্টা করে, যা একটি বড় সংখ্যক ডিসিশন ট্রি তৈরি করে এবং তারপর একটি গড় ফাংশন প্রয়োগ করে একটি উন্নত সামগ্রিক মডেল তৈরি করে।


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

ভালো কাজ 👏!

চলুন একটি Boosted Tree মডেলের সাথেও পরীক্ষা-নিরীক্ষা করি।

Boosted Tree একটি ensemble পদ্ধতি নির্ধারণ করে যা একটি ধারাবাহিক সিদ্ধান্ত গাছের সিরিজ তৈরি করে, যেখানে প্রতিটি গাছ পূর্ববর্তী গাছের ফলাফলের উপর নির্ভর করে এবং ধাপে ধাপে ত্রুটি কমানোর চেষ্টা করে। এটি ভুলভাবে শ্রেণীবদ্ধ আইটেমগুলোর ওজনের উপর মনোযোগ দেয় এবং পরবর্তী শ্রেণীবিন্যাসকারীর জন্য ফিট সামঞ্জস্য করে সেগুলো সংশোধন করে।

এই মডেলটি ফিট করার বিভিন্ন উপায় রয়েছে (দেখুন `help("boost_tree")`)। এই উদাহরণে, আমরা `xgboost` ইঞ্জিনের মাধ্যমে Boosted trees ফিট করব।


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ দয়া করে দেখুন:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - AdaBoost মডেল নিয়ে আলোচনা করে, যা xgboost-এর একটি ভালো বিকল্প।
>
> Ensemble classifiers সম্পর্কে আরও জানতে।

## ৪. অতিরিক্ত - একাধিক মডেলের তুলনা

আমরা এই ল্যাবে বেশ কয়েকটি মডেল ফিট করেছি 🙌। বিভিন্ন সেটের প্রিপ্রসেসর এবং/অথবা মডেল স্পেসিফিকেশন থেকে অনেকগুলো ওয়ার্কফ্লো তৈরি করা এবং তারপর একে একে পারফরম্যান্স মেট্রিক গণনা করা বেশ ক্লান্তিকর বা সময়সাপেক্ষ হতে পারে।

চলুন দেখি আমরা এটি কীভাবে সমাধান করতে পারি একটি ফাংশন তৈরি করে, যা ট্রেনিং সেটে একটি ওয়ার্কফ্লো তালিকা ফিট করে এবং তারপর টেস্ট সেটের উপর ভিত্তি করে পারফরম্যান্স মেট্রিক প্রদান করে। আমরা [purrr](https://purrr.tidyverse.org/) প্যাকেজ থেকে `map()` এবং `map_dfr()` ব্যবহার করব, যা তালিকার প্রতিটি উপাদানে ফাংশন প্রয়োগ করতে সাহায্য করে।

> [`map()`](https://purrr.tidyverse.org/reference/map.html) ফাংশন আপনাকে অনেকগুলো for লুপের পরিবর্তে এমন কোড ব্যবহার করতে দেয়, যা সংক্ষিপ্ত এবং পড়তে সহজ। [`map()`](https://purrr.tidyverse.org/reference/map.html) ফাংশন সম্পর্কে শেখার সেরা জায়গা হলো R for data science-এর [iteration chapter](http://r4ds.had.co.nz/iteration.html)।


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) প্যাকেজটি ব্যবহারকারীদের অনেকগুলো মডেল তৈরি এবং সহজে ফিট করার সুযোগ দেয়, তবে এটি মূলত পুনরাবৃত্তি নমুনা গ্রহণের (resampling techniques) সাথে কাজ করার জন্য ডিজাইন করা হয়েছে, যেমন `cross-validation`, যা আমরা এখনো বিস্তারিত আলোচনা করিনি।

## **🚀চ্যালেঞ্জ**

প্রতিটি পদ্ধতির অনেকগুলো প্যারামিটার থাকে যা আপনি পরিবর্তন করতে পারেন, যেমন SVM-এ `cost`, KNN-এ `neighbors`, এবং র‍্যান্ডম ফরেস্টে `mtry` (Randomly Selected Predictors)।

প্রতিটি মডেলের ডিফল্ট প্যারামিটারগুলো নিয়ে গবেষণা করুন এবং ভাবুন, এই প্যারামিটারগুলো পরিবর্তন করলে মডেলের গুণগত মানের উপর কী প্রভাব পড়তে পারে।

কোনো নির্দিষ্ট মডেল এবং তার প্যারামিটার সম্পর্কে আরও জানতে ব্যবহার করুন: `help("model")`, যেমন `help("rand_forest")`

> বাস্তবে, আমরা সাধারণত এই প্যারামিটারগুলোর *সেরা মান* *অনুমান* করি, যা অনেকগুলো মডেল একটি `সিমুলেটেড ডেটা সেট`-এ প্রশিক্ষণ দিয়ে এবং সেগুলোর কার্যকারিতা পরিমাপ করে করা হয়। এই প্রক্রিয়াটি **টিউনিং** নামে পরিচিত।

### [**পোস্ট-লেকচার কুইজ**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **পুনরালোচনা ও স্ব-অধ্যয়ন**

এই পাঠগুলোতে অনেক নতুন শব্দ রয়েছে, তাই [এই তালিকা](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) থেকে প্রয়োজনীয় পরিভাষাগুলো একবার দেখে নিন!

#### কৃতজ্ঞতা:

[`অ্যালিসন হর্স্ট`](https://twitter.com/allison_horst/) কে ধন্যবাদ, যিনি চমৎকার ইলাস্ট্রেশন তৈরি করেছেন যা R-কে আরও আকর্ষণীয় এবং সহজবোধ্য করে তুলেছে। তার আরও ইলাস্ট্রেশন দেখতে পারেন এই [গ্যালারিতে](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM)।

[Cassie Breviu](https://www.twitter.com/cassieview) এবং [Jen Looper](https://www.twitter.com/jenlooper) কে ধন্যবাদ, যারা এই মডিউলের মূল পাইথন সংস্করণ তৈরি করেছেন ♥️

শুভ শিক্ষণ,

[Eric](https://twitter.com/ericntay), গোল্ড মাইক্রোসফট লার্ন স্টুডেন্ট অ্যাম্বাসেডর।

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>ইলাস্ট্রেশন: @allison_horst</figcaption>



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
